# 01 - Preprocess FIPS Codes for CVD Mortality Data

This notebook merges the CVD mortality data with FIPS codes to enable joining with other datasets (ACS, weather, livestock).

The location names in IHME data are in the format "County Name (State)", which we parse to extract County and State, then merge with the FIPS lookup table.

In [1]:
import pandas as pd
import os

In [2]:
def preprocess_fips_cvd(year):
    """
    Preprocesses and merges FIPS data with CVD mortality data for a given year.
    
    Parameters:
    year (int): The year of the CVD mortality data (e.g., 2012).
    
    Returns:
    pd.DataFrame: Preprocessed DataFrame with CVD mortality and FIPS codes.
    """
    # Fixed file paths
    fips_path = '../data_cvd/raw/state_fips.csv'
    cvd_path = f'../data_cvd/processed/cvd_single_year/cvd_mortality_{year}.csv'

    # Load and preprocess FIPS codes
    fips_df = pd.read_csv(fips_path, dtype={'fips': str})
    fips_df['fips'] = fips_df['fips'].str.zfill(5)  # Ensure all FIPS codes are 5 characters
    fips_df['State_FIPS'] = fips_df['fips'].str[:2]
    fips_df['County_FIPS'] = fips_df['fips'].str[2:]
    fips_df['state_full'] = fips_df['state'].map({
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
        'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
        'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
        'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
        'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
        'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
        'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
        'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
        'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
        'WI': 'Wisconsin', 'WY': 'Wyoming'
    })

    # Load and preprocess CVD mortality data
    cvd_df = pd.read_csv(cvd_path)
    
    # Extract County and State from location_name (format: "County Name (State)")
    cvd_df[['County', 'State']] = cvd_df['location_name'].str.extract(r'^(.*) \((.*)\)$')
    
    # Keep relevant columns and strip whitespace
    cvd_df = cvd_df[['County', 'State', 'cvd_mortality_rate', 'year']].copy()
    cvd_df['County'] = cvd_df['County'].str.strip()
    cvd_df['State'] = cvd_df['State'].str.strip()

    # Merge FIPS and CVD mortality data
    merged_df = pd.merge(
        cvd_df,
        fips_df[['State_FIPS', 'County_FIPS', 'name', 'state_full', 'fips']],
        left_on=['State', 'County'],
        right_on=['state_full', 'name'],
        how='left'
    )

    # Rename fips column to Fips for consistency with other datasets
    merged_df = merged_df.rename(columns={'fips': 'Fips'})
    
    # Drop redundant columns and return
    return merged_df.drop(columns=['name', 'state_full'])

## Test with a Single Year

In [3]:
# Test with 2012
df_test = preprocess_fips_cvd(2012)
print(f"Shape: {df_test.shape}")
print(f"Columns: {df_test.columns.tolist()}")
df_test.head()

Shape: (3127, 7)
Columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fips']


,County,State,cvd_mortality_rate,year,State_FIPS,County_FIPS,Fips
0,Autauga County,Alabama,0.003293,2012,01,001,01001
1,Baldwin County,Alabama,0.002951,2012,01,003,01003
2,Barbour County,Alabama,0.003027,2012,01,005,01005
3,Bibb County,Alabama,0.003566,2012,01,007,01007
4,Blount County,Alabama,0.003056,2012,01,009,01009


In [4]:
# Check for missing FIPS codes
missing_fips = df_test[df_test['Fips'].isna()]
print(f"Rows with missing FIPS: {len(missing_fips)}")
if len(missing_fips) > 0:
    print("Sample missing rows:")
    print(missing_fips[['County', 'State']].head(10))

Rows with missing FIPS: 16
Sample missing rows:
                                          County   State
80                           Chugach Census Area  Alaska
81                      Copper River Census Area  Alaska
83                          Kusilvak Census Area  Alaska
85                             Kobuk Census Area  Alaska
87                  Aleutian Islands Census Area  Alaska
96            Skagway-Yakutat-Angoon Census Area  Alaska
97             Skagway-Hoonah-Angoon Census Area  Alaska
102  Prince of Wales-Outer Ketchikan Census Area  Alaska
104              Wrangell-Petersburg Census Area  Alaska
325                                          NaN     NaN


## Process All Years (2012-2019)

In [5]:
# Ensure output directory exists
output_dir = '../data_cvd/processed/preprocessed_fips_cvd/'
os.makedirs(output_dir, exist_ok=True)

# Process all years
for year in range(2012, 2020):
    df = preprocess_fips_cvd(year)
    df.dropna(inplace=True)
    output_path = f'{output_dir}preprocessed_cvd_fips_{year}.csv'
    df.to_csv(output_path, index=False)
    print(f"Year {year}: {len(df)} rows saved to {output_path}")

Year 2012: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2012.csv
Year 2013: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2013.csv
Year 2014: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2014.csv
Year 2015: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2015.csv
Year 2016: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2016.csv
Year 2017: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2017.csv
Year 2018: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2018.csv
Year 2019: 3111 rows saved to ../data_cvd/processed/preprocessed_fips_cvd/preprocessed_cvd_fips_2019.csv


## Verify Output Files

In [6]:
# List all generated files
files = sorted([f for f in os.listdir(output_dir) if f.endswith('.csv')])
print("Generated files:")
for f in files:
    filepath = os.path.join(output_dir, f)
    df = pd.read_csv(filepath)
    print(f"  {f}: {len(df)} rows, columns: {df.columns.tolist()}")

Generated files:
  preprocessed_cvd_fips_2012.csv: 3111 rows, columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fips']
  preprocessed_cvd_fips_2013.csv: 3111 rows, columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fips']
  preprocessed_cvd_fips_2014.csv: 3111 rows, columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fips']
  preprocessed_cvd_fips_2015.csv: 3111 rows, columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fips']
  preprocessed_cvd_fips_2016.csv: 3111 rows, columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fips']
  preprocessed_cvd_fips_2017.csv: 3111 rows, columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fips']
  preprocessed_cvd_fips_2018.csv: 3111 rows, columns: ['County', 'State', 'cvd_mortality_rate', 'year', 'State_FIPS', 'County_FIPS', 'Fip

In [7]:
# Sample the final output
df_sample = pd.read_csv(f'{output_dir}preprocessed_cvd_fips_2012.csv')
df_sample.head(10)

,County,State,cvd_mortality_rate,year,State_FIPS,County_FIPS,Fips
0,Autauga County,Alabama,0.003293,2012,1,1,1001
1,Baldwin County,Alabama,0.002951,2012,1,3,1003
2,Barbour County,Alabama,0.003027,2012,1,5,1005
3,Bibb County,Alabama,0.003566,2012,1,7,1007
4,Blount County,Alabama,0.003056,2012,1,9,1009
5,Bullock County,Alabama,0.003317,2012,1,11,1011
6,Butler County,Alabama,0.003426,2012,1,13,1013
7,Calhoun County,Alabama,0.003727,2012,1,15,1015
8,Chambers County,Alabama,0.003230,2012,1,17,1017
9,Cherokee County,Alabama,0.003113,2012,1,19,1019
